# Scheduler Local Test


## Setup


* `make start-scheduler` 
* `make start-envoy`
* `make start-agent`
* `make start-mlserver`
* `make start-rclone`

## No Auth Example

In [7]:
!grpcurl -d '{"model":{"name":"iris","version":"2","uri":"gs://seldon-models/mlserver/iris",\
         "requirements":["sklearn"],\
         "memoryBytes":500,\
         "replicas":1}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [8]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [9]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55c8259664f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55c8259664f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Thu, 09 Dec 2021 17:34:06 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1082
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"a76a3462-d283-4f4f-ac9e-e4350016f3bf","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [10]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [11]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "version": "2",
  "serverName": "mlserver",
  "modelReplicaState": {
    "0": {
      "state": "Available",
      "timestamp": "2021-12-09T17:34:04.526879263Z"
    }
  },
  "state": {
    "state": "ModelAvailable",
    "availableReplicas": 1,
    "timestamp": "2021-12-09T17:34:04.526879263Z"
  }
}


In [13]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [6]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [8]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

ERROR:
  Code: FailedPrecondition
  Message: Failed to find model iris


In [9]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "1000"
    }
  ]
}


In [10]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x557ff24f64f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x557ff24f64f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)







* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


## Inline RClone Config Example

Before running this 
  * install minio in a kind cluster with Ansible.
  * Update ip address for exposed minio ip address in example below
  
Create rclone.conf changing the ip address for minio as appropriate

```
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://172.18.255.1:9000
```
  
 Copy iris model to minio
 
 ```
 rclone --config ./rclone.conf copy mlrepo/iris s3://test
 ```

In [11]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
           storageRcloneConfig: "{\"type\":\"s3\",\"name\":\"s3\",\"parameters\":{\"provider\":\"minio\",\"env_auth\":\"false\",\"access_key_id\":\"minioadmin\",\"secret_access_key\":\"minioadmin\",\"endpoint\":\"http://172.18.255.1:9000\"}}" \
     } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [12]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [13]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55989d9b04f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55989d9b04f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Fri, 03 Dec 2021 08:08:47 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1089
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"c0f07181-e262-4c77-8110-67f4f388d18f","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [14]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [15]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


## K8S Secret RClone Config Example

 * Update endpoint for minio below

In [16]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://172.18.255.1:9000

Overwriting minio-secret.yaml


In [17]:
!kubectl apply -f minio-secret.yaml

secret/minio-secret configured


In [18]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
      storageSecretName: "minio-secret" \
      } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [19]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "1000",
      "availableMemoryBytes": "500"
    }
  ]
}


In [20]:
!curl -v http://0.0.0.0:9000/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x555587a654f0)
*   Trying 0.0.0.0...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x555587a654f0)
* Connected to 0.0.0.0 (127.0.0.1) port 9000 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Fri, 03 Dec 2021 08:09:04 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1106
< 
* Connection #0 to host 0.0.0.0 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"016c0054-fcaf-4083-8274-1b589781cc19","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [21]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
        0.0.0.0:9000 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [22]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  0.0.0.0:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}
